In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras import backend as K

Using TensorFlow backend.


In [2]:
# allocates GPU memory based on runtime allocations. doesn't releases memory because of potential memory fragmentation
# without this option, getting images will not work
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [17]:
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 844
nb_validation_samples = 362
epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [4]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [5]:
# this is the augmentation configuration for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [6]:
# this is the augmentation configuration we will use for testing:
test_datagen = ImageDataGenerator(
    rescale=1./255)

In [7]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 844 images belonging to 2 classes.


In [8]:
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 359 images belonging to 2 classes.


In [9]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size
)

model.save_weights('model.h5')

Epoch 1/50
22/22 [==============================] - 5s 217ms/step - loss: 0.7247 - acc: 0.4630 - val_loss: 0.6887 - val_acc: 0.5766
Epoch 2/50
22/22 [==============================] - 3s 121ms/step - loss: 0.6961 - acc: 0.5170 - val_loss: 0.6914 - val_acc: 0.6072
Epoch 3/50
22/22 [==============================] - 3s 130ms/step - loss: 0.6971 - acc: 0.5142 - val_loss: 0.6881 - val_acc: 0.5042
Epoch 4/50
22/22 [==============================] - 3s 128ms/step - loss: 0.6965 - acc: 0.5369 - val_loss: 0.6796 - val_acc: 0.5989
Epoch 5/50
22/22 [==============================] - 3s 127ms/step - loss: 0.6902 - acc: 0.5606 - val_loss: 0.6751 - val_acc: 0.4680
Epoch 6/50
22/22 [==============================] - 3s 127ms/step - loss: 0.6827 - acc: 0.5795 - val_loss: 0.6376 - val_acc: 0.6630
Epoch 7/50
22/22 [==============================] - 3s 129ms/step - loss: 0.6990 - acc: 0.5256 - val_loss: 0.6556 - val_acc: 0.7047
Epoch 8/50
22/22 [==============================] - 3s 129ms/step - loss: 0.

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 64)        18496     
__________

In [11]:
loaded_model = Sequential()

loaded_model.add(Conv2D(32, (3, 3), input_shape=input_shape))
loaded_model.add(Activation('relu'))
loaded_model.add(MaxPooling2D(pool_size=(2, 2)))

loaded_model.add(Conv2D(32, (3, 3)))
loaded_model.add(Activation('relu'))
loaded_model.add(MaxPooling2D(pool_size=(2, 2)))

loaded_model.add(Conv2D(64, (3, 3)))
loaded_model.add(Activation('relu'))
loaded_model.add(MaxPooling2D(pool_size=(2, 2)))

loaded_model.add(Flatten())
loaded_model.add(Dense(64))
loaded_model.add(Activation('relu'))
loaded_model.add(Dropout(0.5))
loaded_model.add(Dense(1))
loaded_model.add(Activation('sigmoid'))

loaded_model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

loaded_model.load_weights('model.h5')

In [12]:
prediction = loaded_model.predict_generator(validation_generator, verbose=1)

23/23 [==============================] - 1s 43ms/step


In [13]:
from sklearn.metrics import confusion_matrix
import numpy as np

y_true = np.array([0] * 207 + [1] * 152)
y_pred = prediction > 0.5

confusion_matrix(y_true, y_pred)

array([[111,  96],
       [ 92,  60]], dtype=int64)

In [34]:
prediction.shape

(359, 1)

In [14]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size
)

model.save_weights('3ConvLayer.h5')

Epoch 1/50
22/22 [==============================] - 3s 154ms/step - loss: 0.7364 - acc: 0.4375 - val_loss: 0.6928 - val_acc: 0.5599
Epoch 2/50
22/22 [==============================] - 3s 129ms/step - loss: 0.6956 - acc: 0.4915 - val_loss: 0.6939 - val_acc: 0.4234
Epoch 3/50
22/22 [==============================] - 3s 135ms/step - loss: 0.6957 - acc: 0.4650 - val_loss: 0.6959 - val_acc: 0.4234
Epoch 4/50
22/22 [==============================] - 3s 134ms/step - loss: 0.6933 - acc: 0.5142 - val_loss: 0.6874 - val_acc: 0.5766
Epoch 5/50
22/22 [==============================] - 3s 136ms/step - loss: 0.6997 - acc: 0.4830 - val_loss: 0.6940 - val_acc: 0.4234
Epoch 6/50
22/22 [==============================] - 3s 130ms/step - loss: 0.6931 - acc: 0.5057 - val_loss: 0.6974 - val_acc: 0.4234
Epoch 7/50
22/22 [==============================] - 3s 129ms/step - loss: 0.6942 - acc: 0.4897 - val_loss: 0.7239 - val_acc: 0.4234
Epoch 8/50
22/22 [==============================] - 3s 135ms/step - loss: 0.

In [15]:
prediction = model.predict_generator(validation_generator, verbose=1)

23/23 [==============================] - 1s 43ms/step


In [16]:
y_true = np.array([0] * 207 + [1] * 152)
y_pred = prediction > 0.5

confusion_matrix(y_true, y_pred)

array([[108,  99],
       [ 75,  77]], dtype=int64)

In [18]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size
)

model.save_weights('1ConvLayer.h5')

Epoch 1/50
52/52 [==============================] - 7s 138ms/step - loss: 7.9138 - acc: 0.4948 - val_loss: 9.1924 - val_acc: 0.4234
Epoch 2/50
52/52 [==============================] - 7s 137ms/step - loss: 5.8671 - acc: 0.4836 - val_loss: 0.7455 - val_acc: 0.4345
Epoch 3/50
52/52 [==============================] - 7s 135ms/step - loss: 0.6881 - acc: 0.5881 - val_loss: 0.6421 - val_acc: 0.6267
Epoch 4/50
52/52 [==============================] - 7s 133ms/step - loss: 0.6597 - acc: 0.6286 - val_loss: 0.5980 - val_acc: 0.6852
Epoch 5/50
52/52 [==============================] - 7s 130ms/step - loss: 0.6473 - acc: 0.6378 - val_loss: 0.6385 - val_acc: 0.6630
Epoch 6/50
52/52 [==============================] - 7s 128ms/step - loss: 0.6246 - acc: 0.6578 - val_loss: 0.6284 - val_acc: 0.6992
Epoch 7/50
52/52 [==============================] - 7s 129ms/step - loss: 0.6110 - acc: 0.6923 - val_loss: 0.6270 - val_acc: 0.7159
Epoch 8/50
52/52 [==============================] - 7s 129ms/step - loss: 0.

In [19]:
prediction = model.predict_generator(validation_generator, verbose=1)

23/23 [==============================] - 1s 53ms/step


In [20]:
y_true = np.array([0] * 207 + [1] * 152)
y_pred = prediction > 0.5

confusion_matrix(y_true, y_pred)

array([[ 96, 111],
       [ 82,  70]], dtype=int64)

In [21]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=250,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size
)

model.save_weights('250epochs.h5')

Epoch 1/250
52/52 [==============================] - 8s 152ms/step - loss: 0.7002 - acc: 0.5164 - val_loss: 0.6987 - val_acc: 0.4234
Epoch 2/250
52/52 [==============================] - 7s 141ms/step - loss: 0.6953 - acc: 0.4964 - val_loss: 0.6926 - val_acc: 0.5515
Epoch 3/250
52/52 [==============================] - 7s 137ms/step - loss: 0.7031 - acc: 0.5240 - val_loss: 0.6845 - val_acc: 0.5766
Epoch 4/250
52/52 [==============================] - 7s 135ms/step - loss: 0.6946 - acc: 0.5232 - val_loss: 0.7155 - val_acc: 0.4234
Epoch 5/250
52/52 [==============================] - 7s 134ms/step - loss: 0.6927 - acc: 0.5393 - val_loss: 0.6752 - val_acc: 0.5933
Epoch 6/250
52/52 [==============================] - 7s 135ms/step - loss: 0.6841 - acc: 0.5889 - val_loss: 0.6350 - val_acc: 0.6964
Epoch 7/250
52/52 [==============================] - 7s 136ms/step - loss: 0.6704 - acc: 0.5849 - val_loss: 0.5820 - val_acc: 0.7075
Epoch 8/250
52/52 [==============================] - 7s 133ms/step - 

In [22]:
prediction = model.predict_generator(validation_generator, verbose=1)

23/23 [==============================] - 1s 42ms/step


In [27]:
y_true = np.array([0] * 207 + [1] * 152)
y_pred = prediction > 0.5

confusion_matrix(y_true, y_pred)

array([[116,  91],
       [ 83,  69]], dtype=int64)